In [2]:
import os
import pandas as pd
import numpy as np

import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

from tqdm import tqdm
import matplotlib.pyplot as plt

<h3>Read raw files</h3>

In [3]:
raw_path = '../data/sas/data.csv'

In [8]:
raw_data = dd.read_csv(
    raw_path,
    sep=';',  
    encoding='cp1251',
    low_memory=False, 
    dtype={
        'ГородМагазина': 'object',
        'МагазинЗаказа': 'object',
        'СуммаДоставки': 'object',
        'СуммаУслуг'   : 'object',
        'ПВЗ_код'      : 'object',
        'ПричинаОтмены': 'object'
    }
)

In [9]:
with ProgressBar():
    data = raw_data.compute()

[########################################] | 100% Completed | 4.59 ss


In [10]:
data.shape

(730558, 38)

In [11]:
data.columns

Index(['Дата', 'ДатаДоставки', 'НомерЗаказаНаСайте', 'НовыйСтатус',
       'СуммаЗаказаНаСайте', 'СуммаДокумента', 'МетодДоставки', 'ФормаОплаты',
       'Регион', 'Группа2', 'Группа3', 'Группа4', 'Тип', 'Номенклатура',
       'ТипТовара', 'Отменено', 'ПричинаОтмены', 'Количество', 'Цена',
       'СуммаСтроки', 'ЦенаЗакупки', 'МесяцДатыЗаказа', 'ГодДатыЗаказа',
       'ПВЗ_код', 'Статус', 'Гео', 'Маржа', 'СуммаУслуг', 'СуммаДоставки',
       'НомерСтроки', 'КоличествоПроданоКлиенту', 'ДатаЗаказаНаСайте',
       'Телефон_new', 'ЭлектроннаяПочта_new', 'Клиент', 'ID_SKU',
       'ГородМагазина', 'МагазинЗаказа'],
      dtype='object')

<h3>Clean</h3>

In [12]:
numeric = ['СуммаЗаказаНаСайте', 
           'СуммаДокумента', 
           'Количество', 
           'Цена',
           'СуммаСтроки', 
           'ЦенаЗакупки',
           'Маржа', 
           'СуммаУслуг',
           'СуммаДоставки', 
           'КоличествоПроданоКлиенту']

def clear_to_float(x):
    return float(str(x).replace(' ', '').replace(',','.'))

for column in numeric:
    data[column] = data[column].apply(clear_to_float)

<h3>Filter data</h3>

In [14]:
assert data['Телефон_new'].isna().sum() == 0

In [15]:
# data with valid user phone number
ph_data = data[data['Телефон_new'].str.contains('[0-9]{8}-[0-9]{14}')] # phone data`

In [16]:
ph_data.shape

(727727, 38)

In [17]:
################################################################################

<h4>Filter region</h4>

In [19]:
TOP_N_REGIONS = 5

regions = ph_data['Регион'].value_counts(normalize=True).round(2)[:TOP_N_REGIONS]
regions

Москва             0.45
Санкт-Петербург    0.06
Нижний Новгород    0.03
Самара             0.02
Екатеринбург       0.02
Name: Регион, dtype: float64

In [20]:
region_data = ph_data[ph_data['Регион'].isin(regions.index)]

In [21]:
################################################################################

<h4>Filter price</h4>

In [22]:
region_data[['Цена', 'СуммаЗаказаНаСайте', 'Количество']].describe().round(2)

,Цена,СуммаЗаказаНаСайте,Количество
count,410210.00,410210.00,410210.00
mean,608.41,4574.76,1.50
std,1589.24,5257.89,2.13
min,0.00,1.00,0.00
25%,81.00,1797.00,1.00
50%,206.00,3127.00,1.00
75%,599.00,5448.00,1.00
max,74990.00,147298.00,150.00


In [23]:
# filter by quantity
pf_data = region_data[region_data['Количество'] != 0]
# filter by price
threshold = pf_data['Цена'].quantile(0.05)
pf_data = pf_data[pf_data['Цена'] > threshold]
print('Remove every order with price lower than', threshold)

Remove every order with price lower than 31.0


In [24]:
pf_data.shape # price filtered data

(379425, 38)

In [25]:
################################################################################

<h4>Filter date</h4>

In [26]:
pf_data['Дата'] = pd.to_datetime(pf_data['Дата'], dayfirst=True)
pf_data['ДатаДоставки'] = pd.to_datetime(pf_data['ДатаДоставки'], dayfirst=True)

In [27]:
print(pf_data['Дата'].min().date().strftime('%d/%m/%Y'), '->', pf_data['Дата'].max().date().strftime('%d/%m/%Y'))

01/05/2017 -> 30/06/2017


In [28]:
print(pf_data['ДатаДоставки'].min().date().strftime('%d/%m/%Y'), '->', pf_data['ДатаДоставки'].max().date().strftime('%d/%m/%Y'))

02/05/2017 -> 29/07/2017


In [29]:
pf_data.groupby(pd.Grouper(key='ДатаДоставки', freq='M'))['Дата'].count()

ДатаДоставки
2017-05-31    168581
2017-06-30    182777
2017-07-31     28067
Freq: M, Name: Дата, dtype: int64

In [36]:
# time filtered data
tf_data = pf_data[pf_data['ДатаДоставки'] > pd.Timestamp(year=2017, month=5, day=1)]

In [37]:
################################################################################

<h3>Item parameters</h3>

In [40]:
tf_data['Группа2'].value_counts().head(4)

ДЕТСКОЕ ПИТАНИЕ       63722
ИГРУШКИ               63639
ТЕКСТИЛЬ, ТРИКОТАЖ    63472
КОСМЕТИКА/ГИГИЕНА     33639
Name: Группа2, dtype: int64

In [41]:
tf_data['Группа3'].value_counts().head(4)

ПЮРЕ                                  35035
ПОДГУЗНИКИ                            29723
ИГРУШКИ ДЛЯ РАЗВИТИЯ МАЛЫШЕЙ          21263
ОДЕЖДА ДЛЯ НОВОРОЖДЕННЫХ (0-2 лет)    20676
Name: Группа3, dtype: int64

In [42]:
tf_data['Группа4'].value_counts().head(4)

трусики-подгузники    14564
подгузники            13460
пюре мясные           12622
пюре фруктовые        10852
Name: Группа4, dtype: int64

In [43]:
tf_data['Тип'].value_counts()

ППКП       150318
ОДЕЖДА      80928
ИГРУШКИ     73336
ИНОЕ        59279
КГТ         15564
Name: Тип, dtype: int64

In [44]:
tf_data['ТипТовара'].value_counts()

МГТ    308349
КГТ     15525
Name: ТипТовара, dtype: int64

In [45]:
(tf_data['ID_SKU'].value_counts() > 70).sum()

520

<h3>DataFrame</h3>

In [46]:
from pathlib import Path

In [47]:
Path("../data/sas/").mkdir(parents=True, exist_ok=True)

In [48]:
df = tf_data[['Дата', 'ID_SKU', 'Телефон_new', 'Цена', 'Количество', 'Группа2', 'Группа3', 'Группа4', 'Тип', 'ТипТовара']]

In [53]:
df.to_csv('../data/sas/positions.csv', index=False)

In [109]:
################################################################################

In [110]:
tf_data.to_parquet('./data/clean/all_positions.parquet')